# Motivation

We thought it would be interesting to analyse the sentiment of Amazon reviews, since reviews often are highly subjective and descriptive. The datasets we chose to work contain both Amazon product reviews and also metadata about each products, such as its related products. This gives us the opportunity to construct and analyse social networks.

The goal is to better understand reviewers, and how they 

# Summary statistics

# Tools, theory and analysis

# Discussion